## Get a list of foods

In [ ]:
import pandas as pd
import time
import random

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys




In [ ]:
# 1. Abre navegador en blanco
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)


# 2. Abrimos web mercadona
URL ="https://tools.myfooddata.com/nutrient-ranking-tool/Water/Vegetarian/Highest/100g/Common/No"
driver.get(URL)




In [ ]:

# Aceptamos cookies
cookies = driver.find_element(By.CLASS_NAME,"mat-focus-indicator solo-button mat-button mat-button-base mat-raised-button")
cookies.send_keys(Keys.ENTER)

In [ ]:
import pandas as pd
import time
import random

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from IPython.display import display
from datetime import datetime


# 1. Abre navegador en blanco
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 2. Abrimos web mercadona
url_mercadona ="https://tienda.mercadona.es/categories/"
driver.get(url_mercadona)

# Esperamos unos segundillos
time.sleep(3)

# Aceptamos cookies
cookies = driver.find_element(By.CLASS_NAME,"ui-button.ui-button--small.ui-button--primary.ui-button--positive")
cookies.send_keys(Keys.ENTER)

# Buscamos por clase el campo de input para poder introducir nuestro código postal
input = driver.find_element(By.CLASS_NAME,"ym-hide-content")

# Introducimos el código postal
input.send_keys("28016")

# Esperamos unos segundillos
time.sleep(round(random.uniform(0.5,3),1))

# Le damos a ENTER
input.send_keys(Keys.ENTER)   #driver.find_element_by_id("submit").click()



time.sleep(5)



dict_mercadona = {"Group":[],"Category":[],"Name":[],"Size":[],"Price":[], "Image":[]}

categories_length = len(driver.find_elements(By.CLASS_NAME,"category-menu__item"))+1
categories_range = range(1,categories_length)

categories = [] # Títulos de las categorías
sub_header_headers= [] # Títulos de los sub-headers
sub_category= []
empty =[]

category_contador = 0
group_contador = 0
sub_category_contador = 0

def get_product_details(categories,empty,group_contador,category_contador):
    # ONCE IN PRODUCT DISPLAY => RECORRER CADA PRODUCTO Y PINCHAR EN ÉL
    contenedor = driver.find_element(By.CLASS_NAME,"category-detail__content")
    contenedores_grupo = contenedor.find_elements(By.CLASS_NAME,"product-container")

    list_products = []
    list_empty = []
    
    for contenedor in contenedores_grupo:
        
        product_container = contenedor.find_elements(By.CLASS_NAME,"product-cell")
        
        for product in product_container:
            product_image = contenedor.find_element(By.TAG_NAME,"img").get_attribute("src")
            product_array = product.text.split("\n")[:-1]
            product_array.append(product_image)
            
            # ADD ALL PRODUCT INFO TO EMPTY ARRAY
            dict_mercadona["Name"].append(product_array[0])
            dict_mercadona["Size"].append(product_array[1])
            dict_mercadona["Price"].append(product_array[2])
            dict_mercadona["Image"].append(product_array[-1])
            dict_mercadona["Group"].append(categories[group_contador])
            dict_mercadona["Category"].append(empty[category_contador])
            

            list_empty.append(product_array)

        list_products.append(list_empty)
        list_empty = []


for num in categories_range:
    
    # Nivel 1 => GROUP:
    header_a_pinchar = driver.find_elements(By.CSS_SELECTOR,f"#root > div.grid-layout > div.grid-layout__sidebar > ul > li:nth-child({num}) > div > button")    
    categories.append(header_a_pinchar[0].text)
    header_a_pinchar[0].click()
    time.sleep(1)

    # Nivel 2 => CATEGORÍAS:
    sub_category_length = len(driver.find_elements(By.CSS_SELECTOR,"#root > div.grid-layout > div.grid-layout__sidebar > ul > li.category-menu__item.open > div > ul")[0].text.split("\n"))+1
    number_of_sub_headers = range(1,sub_category_length)


    # Pincha todos los botones sub headers y almacena los títulos
    for i in number_of_sub_headers:

        sub_header_button = driver.find_elements(By.CSS_SELECTOR,f"#root > div.grid-layout > div.grid-layout__sidebar > ul > li.category-menu__item.open > div > ul > li:nth-child({i}) > button")        
        empty.append(sub_header_button[0].text)
        sub_header_button[0].click()
        time.sleep(1)
        # Una vez en el bloque adecuado cogemos todos los productos
        get_product_details(categories,empty,group_contador,category_contador)
        category_contador+=1
        sub_category_contador+=1

    group_contador+=1
    
    # sub_header_headers.append(empty)
    # empty =[]
     


df_mercadona = pd.DataFrame(dict_mercadona)

df_mercadona



# Add columns: Name and Date 
df_mercadona['Shop'] = 'Mercadona' 
today = pd.Timestamp.now()
df_mercadona['Date'] = today 
print(today)
df_mercadona.head()
